In [16]:
import jax
import jax.numpy as jnp
from functools import partial


In [17]:
def f(x,val: int):
    print("type of x",type(x))
    print("type of val",type(val))
    return x.sum() + val




In [18]:
jitted_f = jax.jit(f,static_argnames=["val"])

In [19]:
out = jitted_f(jnp.array([1, 2, 3]), 10)

type of x <class 'jax._src.interpreters.partial_eval.DynamicJaxprTracer'>
type of val <class 'int'>


In [22]:
out.item()


16